In [ ]:
import random


jobs_data = {
    1: [(1, 2), (2, 3), (3, 2)],
    2: [(2, 4), (3, 1), (1, 3)],
    3: [(3, 2), (1, 3), (2, 2)],
    4: [(1, 4), (2, 3)],
    5: [(2, 2), (3, 3), (1, 1), (2, 2)],
}


population_size = 20
generations = 100
mutation_rate = 0.1
crossover_rate = 0.8


def initialize_population(jobs_data, population_size):
    population = []
    jobs = list(jobs_data.keys())
    for _ in range(population_size):
        schedule = jobs[:]
        random.shuffle(schedule)
        population.append(schedule)
    return population


def calculate_makespan(schedule, jobs_data):
    machine_end_times = {1: 0, 2: 0, 3: 0}
    job_end_times = {job: 0 for job in jobs_data}
    
    for job in schedule:
        operations = jobs_data[job]
        for machine, duration in operations:
            start_time = max(job_end_times[job], machine_end_times[machine])
            end_time = start_time + duration
            machine_end_times[machine] = end_time
            job_end_times[job] = end_time
    
    return max(job_end_times.values())


def selection(population, fitnesses):
    total_fitness = sum(fitnesses)
    pick = random.uniform(0, total_fitness)
    current = 0
    for individual, fitness in zip(population, fitnesses):
        current += fitness
        if current > pick:
            return individual


def crossover(parent1, parent2):
    if random.random() < crossover_rate:
        size = len(parent1)
        cx_point1 = random.randint(0, size - 1)
        cx_point2 = random.randint(0, size - 1)
        
        if cx_point1 > cx_point2:
            cx_point1, cx_point2 = cx_point2, cx_point1

        child1 = parent1[:cx_point1] + parent2[cx_point1:cx_point2] + parent1[cx_point2:]
        child2 = parent2[:cx_point1] + parent1[cx_point1:cx_point2] + parent2[cx_point2:]
        
        return child1, child2
    else:
        return parent1[:], parent2[:]


def mutate(schedule):
    if random.random() < mutation_rate:
        idx1 = random.randint(0, len(schedule) - 1)
        idx2 = random.randint(0, len(schedule) - 1)
        schedule[idx1], schedule[idx2] = schedule[idx2], schedule[idx1]


def genetic_algorithm(jobs_data, population_size, generations):
    population = initialize_population(jobs_data, population_size)
    
    for generation in range(generations):
        fitnesses = [1 / calculate_makespan(schedule, jobs_data) for schedule in population]
        
        new_population = []
        while len(new_population) < population_size:
            parent1 = selection(population, fitnesses)
            parent2 = selection(population, fitnesses)
            child1, child2 = crossover(parent1, parent2)
            mutate(child1)
            mutate(child2)
            new_population.append(child1)
            new_population.append(child2)
        
        population = new_population[:population_size]
    
    best_schedule = min(population, key=lambda x: calculate_makespan(x, jobs_data))
    best_makespan = calculate_makespan(best_schedule, jobs_data)
    
    return best_schedule, best_makespan


best_schedule, best_makespan = genetic_algorithm(jobs_data, population_size, generations)
print(f"Best makespan: {best_makespan}")
print(f"Best schedule: {best_schedule}")

for job in best_schedule:
    operations = jobs_data[job]
    for machine, duration in operations:
        print(f"Job {job} on Machine {machine} for {duration} hours")